# Auswertung_a

### README

###### Tool zur Auswertung des Typenbestands und Typenaffinität in Form von Graphen

Die HTML-Datei, die denselben Namen wie die Variable dateiname trägt, befindet sich im Basisordner (diese Hinweis tritt üblicherweise bei der Ausführung des Codes in der Google Colab-Anwendung auf).

##### Anpassbare Variablen:

Variable dateiname: Diese Variable bestimmt den Namen der generierten HTML-Graphendatei. Fülle sie entsprechend deinen Anforderungen aus.

Variable haeufigkeit: Diese Variable legt fest, wie oft die Typen miteinander kombiniert sein müssen, um in die Analyse aufgenommen zu werden.


In [7]:
dateiname = "auswertung_a"
haeufigkeit = 1

In [8]:
import xml.etree.ElementTree as ET  # Importieren der ElementTree-Bibliothek zum Parsen von XML

# XML-Daten laden
tree = ET.parse('Textkorpus.xml')
root_node = tree.getroot()

# Namespace definieren
NS = {'tei': 'http://www.tei-c.org/ns/1.0'}

# Filtereinstellung für das Attribut `@ident` (kann leer sein)
# filter_ident = ""  # z. B. input("Bitte geben Sie das Attribut 'ident' an (z.B. 'kat'): ").strip()

# Variable für die gesammelten Märchendaten
maerchen = ""

# Iteration durch die XML-Daten
for corp in root_node.findall(".//tei:teiCorpus", NS):
    if '' in corp.attrib.get("n", ""):# or "xtk" in corp.attrib.get("n", ""):
        for tei in corp.findall("tei:TEI", NS):
            # Filter basierend auf `@ident` oder bei leerem `filter_ident`
#             language = tei.find(".//tei:teiHeader//tei:langUsage/tei:language", NS)
#             if language is not None:
#                 ident_value = language.attrib.get('ident', '')
#                 if filter_ident == "" or ident_value == filter_ident:
                    for ganze in tei.findall(".//tei:text", NS):
                        if '' in ganze.attrib['{http://www.w3.org/XML/1998/namespace}id']:
                           #or 'cr_' in ganze.attrib['{http://www.w3.org/XML/1998/namespace}id']): 
                            quelle = ganze.attrib.get('{http://www.w3.org/XML/1998/namespace}id', '')
                            for body in ganze.findall(".//tei:body", NS):
                                for absatz in body.findall(".//tei:p", NS):
                                    for phrase in absatz.findall(".//tei:seg", NS):
                                        # Labels extrahieren
                                        labelx = ganze.attrib['{http://www.w3.org/XML/1998/namespace}id']
                                        labela = phrase.attrib['{www.dglab.uni-jena.de/vmf/a}ana']
                                        if labela.startswith ('a') and labela !="aU":

                                            maerchen += labela+','+labelx+'\n'
                                
with open("anzahl.csv", 'w', encoding='utf-8') as f:
    f.write ('labela,labelx\n')
    f.write (maerchen)

In [9]:
import xml.etree.ElementTree as ET
import pandas as pd
from pyvis.network import Network
from itertools import combinations
from collections import defaultdict

# CSV-Datei einlesen
df = pd.read_csv('anzahl.csv')

# Duplikate entfernen, um einzigartige Datensätze zu erhalten
unique_df = df.drop_duplicates()

# Netzwerk erstellen
net = Network(notebook=True)
net.force_atlas_2based()

# Dictionary zur Speicherung der Kanten, deren Gewicht und der zugehörigen labelx-Werte
edges = {}
min_frequency = 1
min_unique_labelx = haeufigkeit
self_loops = {}
node_frequency = defaultdict(set)

# Schritt 1: Parsen der XSD-Datei
xsd_path = 'kf/vmf_a.xsd'
tree = ET.parse(xsd_path)
root = tree.getroot()

# Namespace für XSD
ns = {'xs': 'http://www.w3.org/2001/XMLSchema'}

# Dictionary zur Speicherung der ersetzten Werte
replacement_dict = {}

# Value-Werte mit Leerzeichen extrahieren und speichern
for enumeration in root.findall(".//xs:enumeration", ns):
    value = enumeration.attrib.get('value', '')
    if ' ' in value:  # Werte mit Leerzeichen
        split_value = value.split(' ', 1)  # Split bei erstem Leerzeichen
        key = split_value[0]
        full_value = value
        replacement_dict[key] = full_value

# Gruppieren nach labelx und zählen, wie viele einzigartige labelx-Werte mit jedem labela verbunden sind
for labelx, group in unique_df.groupby('labelx'):
    labelas = group['labela'].unique()
    
    if len(labelas) == 1:
        labela = labelas[0]
        if labela in self_loops:
            self_loops[labela]['weight'] += 1
            self_loops[labela]['labelx_values'].append(labelx)
        else:
            self_loops[labela] = {'weight': 1, 'labelx_values': [labelx]}
        node_frequency[labela].add(labelx)
    else:
        for pair in combinations(labelas, 2):
            sorted_pair = tuple(sorted(pair))
            if sorted_pair in edges:
                edges[sorted_pair]['weight'] += 1
                edges[sorted_pair]['labelx_values'].append(labelx)
            else:
                edges[sorted_pair] = {'weight': 1, 'labelx_values': [labelx]}
            node_frequency[pair[0]].add(labelx)
            node_frequency[pair[1]].add(labelx)

# Berechne die Häufigkeit basierend auf der Anzahl der einzigartigen labelx-Werte
node_frequency = {node: len(labelx_set) for node, labelx_set in node_frequency.items()}

# Funktion zur Skalierung der Knotengröße
def get_node_size(frequency, min_size=10, max_size=50):
    max_frequency = max(node_frequency.values()) if node_frequency else 1
    return min_size + (max_size - min_size) * (frequency / max_frequency)

# Schritt 3: Knoten und Kanten hinzufügen
for (labela1, labela2), data in edges.items():
    if data['weight'] >= min_frequency:
        if node_frequency[labela1] >= min_unique_labelx and node_frequency[labela2] >= min_unique_labelx:
            # Ersetzung im Titel, falls im replacement_dict vorhanden
            title1 = replacement_dict.get(labela1, labela1)
            title2 = replacement_dict.get(labela2, labela2)
            
            size1 = get_node_size(node_frequency[labela1])
            size2 = get_node_size(node_frequency[labela2])
            
            net.add_node(labela1, labela1, title=f'{title1} ({node_frequency[labela1]})', size=size1)
            net.add_node(labela2, labela2, title=f'{title2} ({node_frequency[labela2]})', size=size2)
            
            # Gewicht der Kante aus dem Dictionary `edges`
            net.add_edge(labela1, labela2, value=data['weight'], title=f'{data["weight"]}')

# Selbstverbindungen hinzufügen
for labela, data in self_loops.items():
    if node_frequency[labela] >= min_unique_labelx:
        title = replacement_dict.get(labela, labela)
        size = get_node_size(node_frequency[labela])
        net.add_node(labela, labela, title=f'{title} ({node_frequency[labela]})', size=size)
        
        # Gewicht der Selbstverbindung
        net.add_edge(labela, labela, value=data['weight'], title=f'{data["weight"]}')

# Graph anzeigen
net.show(dateiname+'.html')

auswertung_a.html
